# Big Data Exam

### Import Packages

In [59]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [60]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra and dropping the keyspace if it exists

In [61]:
session.execute("DROP KEYSPACE IF EXISTS nmt")

### Creating the keyspace if it doesn't exist

In [62]:
session.execute("CREATE KEYSPACE IF NOT EXISTS nmt WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

### Listing out all the keyspaces and using the above created keyspace is present in the list below and selecting the keyspace which is created by us.

In [63]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
nmt
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Dropping the user table if exists in order to avoid any discrepancy

In [64]:
session.execute("""DROP TABLE IF EXISTS nmt.Users""")

### Creating a user table with attributes user_id, date, file_type, file and the user_id as a PRIMARY KEY

In [65]:
session.execute("""
CREATE TABLE IF NOT EXISTS nmt.Users ( 
    user_id INT, 
    date VARCHAR, 
    file_type TEXT, 
    file TEXT,
    PRIMARY KEY(user_id)
);
""")

### Loading the user data from the users csv

In [66]:
df = pd.read_csv('users.csv')
df

,user_id,date,file_type,file
0,1000,'10/04/2023','txt','Project Requirements'
1,1001,'10/04/2023','txt','Specification'
2,1002,'10/04/2023','txt','Deployment Testing'


### Lopping over the data and inserting it one by one into the dataframe

In [67]:
for index, row in df.iterrows():
    print(f"user_id = {row.user_id}, date ={row.date}, file_type = {row.file_type}, file = {row.file}")

user_id = 1000, date ='10/04/2023', file_type = 'txt', file = 'Project Requirements'
user_id = 1001, date ='10/04/2023', file_type = 'txt', file = 'Specification'
user_id = 1002, date ='10/04/2023', file_type = 'txt', file = 'Deployment Testing'


### INSERT: Displaying the queries and Inserting the data into the table from the dataframe above

In [68]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO nmt.Users (user_id, date, file_type, file)     
        VALUES ({row.user_id}, {row.date}, {row.file_type}, {row.file});
        """
    )
    session.execute(f"""
        INSERT INTO nmt.Users (user_id, date, file_type, file)      
        VALUES ({row.user_id}, {row.date}, {row.file_type}, {row.file});
        """
    )


        INSERT INTO nmt.Users (user_id, date, file_type, file)     
        VALUES (1000, '10/04/2023', 'txt', 'Project Requirements');
        

        INSERT INTO nmt.Users (user_id, date, file_type, file)     
        VALUES (1001, '10/04/2023', 'txt', 'Specification');
        

        INSERT INTO nmt.Users (user_id, date, file_type, file)     
        VALUES (1002, '10/04/2023', 'txt', 'Deployment Testing');
        


### SELECT : Displaying all the users data by querying the table below

In [69]:
rows = session.execute("select (user_id, date, file_type, file) from nmt.Users")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, file_type={row[0][2]}, file={row[0][3]}")


user_id=1001, date=10/04/2023, file_type=txt, file=Specification
user_id=1002, date=10/04/2023, file_type=txt, file=Deployment Testing
user_id=1000, date=10/04/2023, file_type=txt, file=Project Requirements


### SELECT : Selecting the data by filtering the data based on the attribute user_id using ALLOW FILTERING

In [70]:
rows = session.execute("select (user_id, date, file_type, file) from nmt.users where user_id = 1000 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

1000, 10/04/2023, txt, Project Requirements


### SELECT : In order to filter the data we have to use either ALLOW FILTERING OR we have to create a PRIMARY KEY or the Index on the column which we are filtering in the query. In our case, user_id is PRIMARY KEY so there is no need to use ALLOW FILTERING

In [71]:
rows = session.execute("select (user_id, date, file_type, file) from nmt.users where user_id = 1000")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

1000, 10/04/2023, txt, Project Requirements


### Update : Updating the date based on the user_id

In [72]:
session.execute("update nmt.users set date='11/04/2023' where user_id = 1002")
rows = session.execute("select (user_id, date, file_type, file) from nmt.users")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

1001, 10/04/2023, txt, Specification
1002, 11/04/2023, txt, Deployment Testing
1000, 10/04/2023, txt, Project Requirements


### Deleting the data with the user_id 1002 and displaying the entire list after successful deletion

In [73]:
session.execute("delete from nmt.users where user_id = 1002")
rows = session.execute("select (user_id, date, file_type, file) from nmt.users")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

1001, 10/04/2023, txt, Specification
1000, 10/04/2023, txt, Project Requirements


### Reading and Getting the data from the text file which is 1MB in size and storing and displaying the data from the text file in a variable.

In [74]:
with open("sanple_text.txt", "r") as fin:
    file_data = fin.read() 
file_data

'I am Naveen and I am studying Masters in Business Analytics and Information Systems and also working as a graduate instructional assistant for one of the course in MUMA College of BusinessI am Naveen and I am studying Masters in Business Analytics and Information Systems and also working as a graduate instructional assistant for one of the course in MUMA College of BusinessI am Naveen and I am studying Masters in Business Analytics and Information Systems and also working as a graduate instructional assistant for one of the course in MUMA College of BusinessI am Naveen and I am studying Masters in Business Analytics and Information Systems and also working as a graduate instructional assistant for one of the course in MUMA College of BusinessI am Naveen and I am studying Masters in Business Analytics and Information Systems and also working as a graduate instructional assistant for one of the course in MUMA College of BusinessI am Naveen and I am studying Masters in Business Analytics

### Getting the file name and file type for the specific user.

In [75]:
rows = session.execute("select (file_type, file) from nmt.users WHERE user_id = 1000")
file_type = rows[0][0][0]
file_name = rows[0][0][1]

/tmp/ipykernel_112945/1806329254.py:2: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  file_type = rows[0][0][0]
/tmp/ipykernel_112945/1806329254.py:3: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  file_name = rows[0][0][1]


### Creation of the new file with the content taken from previous file and naming the file as a name which is taken from the data frame for a specific user

In [76]:
with open(file_name+'.'+file_type, "w") as fout:
    fout.write(file_data)

### I have selected Cassandra DB as a preferred database because it handles large amounts of data with high write and read throughput. It is also distributed and horizontally scalable architecture. It also handles high availability and fault tolerance. Also There is no Single Point of Failure.

### Due to the absence of Master Node in the Cassandra DB, There is a high availability.
### It has the capability of handling many users and also they can generate the file by providing the content and the file name above.